# COCO Test

In [1]:
# %pip install coco-experiment cocopp modcma numpy pandas

In [81]:
import cocoex  # experimentation module
import cocopp  # post-processing module (not strictly necessary)
import modcma  # ModCMA as solver
import numpy as np
import scipy.optimize
import pandas as pd

In [3]:
import warnings
# Filter out unwanted termination criteria warnings from modcma
# warnings.filterwarnings(action = "ignore", message = "Termination criteria met.*", module = "modcma")
warnings.filterwarnings('ignore')

In [115]:
def build_schedule(solved_per_problem, evals_per_problem, max_iters = 100, log = True):
    assert(len(solved_per_problem) == len(evals_per_problem))

    prior_likelihood = np.ones(len(solved_per_problem)) / len(solved_per_problem)
    expected_evals = np.zeros(len(solved_per_problem))
    
    schedule = []

    for _ in range(0, max_iters):
        # solve_per_eval = (solved_per_problem.mul(prior_likelihood, axis=0).mean(axis=0) / evals_per_problem.mul(prior_likelihood, axis=0).mean(axis=0))
        solve_per_eval = (solved_per_problem / evals_per_problem).mul(prior_likelihood, axis=0).mean(axis=0)
        idxmax = solve_per_eval.idxmax()

        schedule.append(idxmax)
        expected_evals += evals_per_problem[idxmax].mul(prior_likelihood, axis=0).values

        prior_likelihood = (1 - solved_per_problem[idxmax]).values * prior_likelihood
        prior_likelihood = prior_likelihood / sum(prior_likelihood)

        if log: print(f"choose: {idxmax}, unsolved: {prior_likelihood.sum():.2f}, expected_evals: {expected_evals.mean():.2f}")
    
    return (schedule, prior_likelihood, expected_evals)

In [116]:
def get_schedule(dim, max_iters = 1000, exclude_fid = None):
    evals_per_problem = pd.read_csv(f"schedule-data/evals_per_problem-{dim}D.csv", index_col="problem_id")
    solved_per_problem = pd.read_csv(f"schedule-data/solved_per_problem-{dim}D.csv", index_col="problem_id")

    if exclude_fid is not None:
        filter_string = f"f{exclude_fid:02}-"

        all_but_fi = [x for x in solved_per_problem.index if not x.startswith(filter_string)]

        evals_per_problem = evals_per_problem.loc[all_but_fi]
        solved_per_problem = solved_per_problem.loc[all_but_fi]

    schedule, _, _ = build_schedule(solved_per_problem, evals_per_problem, max_iters, log = False)
    return schedule

In [117]:
# Schedule Setup

schedule_name = "GRS-LOPO"

# schedule_name = "Powell"
# config_schedule = ["Powell"]

In [120]:
### prepare
suite_name = "bbob"
dimension = "2,3,5,10"

suite = cocoex.Suite(suite_name, "instances: 1-15", f"dimensions: {dimension}")  # see https://numbbo.github.io/coco-doc/C/#suite-parameters
output_folder = f'{schedule_name}'
observer = cocoex.Observer(suite_name, "result_folder: " + output_folder)

minimal_print = cocoex.utilities.MiniPrint()

COCO INFO: Results will be output to folder exdata/GRS-LOPO


In [121]:
for problem in suite:  # solve all selected problems
    dim = problem.dimension
    budget = 100_000 * dim

    lambda_default = int(4 + np.floor(3 * np.log(dim))) # default

    cma_configurations = {
        "default": {},
        "2L": {"lambda_": 2 * lambda_default},
        "4L": {"lambda_": 4 * lambda_default},
        "8L": {"lambda_": 8 * lambda_default},
        "16L": {"lambda_": 16 * lambda_default},
        "32L": {"lambda_": 32 * lambda_default},
        "64L": {"lambda_": 64 * lambda_default},
        # "active": {"active": True},
        # "elitist": {"elitist": True},
        # "orthogonal": {"orthogonal": True},
        # "sequential": {"sequential": True},
        # "threshold_convergence": {"threshold_convergence": True},
        # "base_sampler_sobol": {"base_sampler": "sobol"},
        # "mirrored": {"mirrored": "mirrored"},
        # "mirrored_pairwise": {"mirrored": "mirrored pairwise"},
    }

    scipy_configurations = {
        "SLSQP": {"method": "SLSQP"},
        "L-BFGS-B": {"method": "L-BFGS-B"},
        "Powell": {"method": "Powell"},
    }

    fid, dim, iid = problem.id_triple

    config_schedule = get_schedule(problem.dimension, exclude_fid = fid)

    problem.observe_with(observer)  # generate data for cocopp
    problem(problem.dimension * [0])  # for better comparability

    restarts = 0

    np.random.seed(0xC0FFEE)

    # while not (problem.final_target_hit == 1 or problem.evaluations > budget):
    while not (problem.final_target_hit == 1 or problem.evaluations > 10 * budget):

        param_id = config_schedule[restarts % len(config_schedule)]
        if restarts >= 1000 and restarts % 1000 == 0:
            print(restarts)
        
        if param_id in cma_configurations.keys():
            params = cma_configurations[param_id]

            # cma = modcma.AskTellCMAES(dim, budget = budget, compute_termination_criteria = True, **params)
            cma = modcma.ModularCMAES(problem, x0 = problem.initial_solution_proposal(), d = dim, budget = budget, compute_termination_criteria = True, **params)

            while not (any(cma.break_conditions) or
                    any(cma.parameters.termination_criteria.values()) or
                    problem.final_target_hit == 1):        
                cma.step()
        
        else:
            params = scipy_configurations[param_id]
            scipy.optimize.minimize(problem, x0 = problem.initial_solution_proposal(), **params)
        
        restarts += 1
        observer.signal_restart(problem)

    minimal_print(problem)  # show progress

2D 21h34:29
 f1||||||||||||||| f2||||||||||||||| f3||||||||||||||| f4|||||||||||||||
 f5||||||||||||||| f6||||||||||||||| f7||||||||||||||| f8|||||||||||||||
 f9|||||||||||||||f10|||||||||||||||f11|||||||||||||||f12|||||||||||||||
f13|||||||||||||||f14|||||||||||||||f15|||||||||||||||f16|||||||||||||||
f17|||||||||||||||f18||||||||||||||1000
|f19|||||||||||||||f20|||||||||||||||
f21|||||||||||||||f22|||||||||||||||f23|||1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
.|1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
.1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
.1000
||1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
.||1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
.|1000
|f24|||||||||||||||
3D 21h39:10
 f1||||||||||||||| f2||||||||||||||| f3||||||||||||||| f4|||||||||||||||
 f5||||||||||||||| f6||||||||||||||| f7||||||||||||||| f8|||||||||||||||
 f9|||||||||||||||f10|||||||||||||||f11|||||||||||||||f12|||||||||||||||
f

In [27]:
restarts

2424

In [122]:
observer.result_folder

'exdata/GRS-LOPO'

In [209]:
### post-process data
# cocopp.main(["--include-fonts", observer.result_folder, "exdata/GRS", "VBS", "bipop-cma!", "HCMA!"]);  # re-run folders look like "...-001" etc
cocopp.main(["--include-fonts", "VBS", "exdata/GRS", "exdata/GRS-LOPO"]);  # re-run folders look like "...-001" etc
# cocopp.main('exdata/Powell exdata/SLSQP exdata/L-BFGS-B exdata/default exdata/2L exdata/4L exdata/8L exdata/16L exdata/32L exdata/64L')

Post-processing (2+)
  Using 3 data sets:
    VBS
    exdata/GRS
    exdata/GRS-LOPO

Post-processing (2+)
  loading data...
  Data consistent according to consistency_check() in pproc.DataSet
  using: VBS
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/GRS
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/GRS-LOPO
  Will generate output data in folder ppdata/VBS_GRS_GRS-L_71c4
    this might take several minutes.
ECDF graphs per noise group...
Loading best algorithm data from refalgs/best2009-bbob.tar.gz ...
  using pickled DataSetList   using: /Users/lennartschapermeier/source/research/cma-rsmp/.conda/lib/python3.12/site-packages/cocopp/refalgs/best2009-bbob.tar.gz
  done (Thu Jan 30 10:22:44 2025).
  done (Thu Jan 30 10:22:47 2025).
ECDF graphs per function group...
  done (Thu Jan 30 10:22:54 2025).
ECDF graphs per function...
  done (Thu Jan 30 10:23:26 2025).
Generating comparison tables...
  done (Thu Jan 30 1

## Performance Assessment

In [210]:
all_algos = ["VBS", "exdata/GRS", "exdata/GRS-LOPO", "exdata/Powell", "exdata/SLSQP", "exdata/L-BFGS-B", "exdata/default", "exdata/2L", "exdata/4L", "exdata/8L", "exdata/16L", "exdata/32L", "exdata/64L"]

In [211]:
# datasets = cocopp.load(cocopp.archives.bbob.get_first("BIPOP"))
# datasets = cocopp.load(cocopp.archives.bbob.get_first("HCMA"))
datasets = cocopp.load(all_algos)

  Data consistent according to consistency_check() in pproc.DataSet


In [212]:
targets = np.power(10, np.linspace(2, -8, 51))

In [213]:
dfs = []

for dset in datasets:
    erts = dset.detERT(targets)
    fid = dset.funcId
    algorithm = dset.algId
    dim = dset.dim

    df = pd.DataFrame({"target": np.log10(targets), "ert": erts})
    df["fid"] = fid
    df["dim"] = dim
    df["algorithm"] = algorithm
    dfs.append(df)

all_ert = pd.concat(dfs)

In [214]:
all_ert.to_csv("all_ert.csv")

0            1.0
1            1.0
2            1.0
3            1.0
4            2.0
         ...    
46    10000000.0
47    10000000.0
48    10000000.0
49    10000000.0
50    10000000.0
Name: ert, Length: 58752, dtype: float64

In [131]:
datasets

[DataSet(portfolio-best on f1 2-D),
 DataSet(GRS on f1 2-D),
 DataSet(Powell on f1 2-D),
 DataSet(SLSQP on f1 2-D),
 DataSet(L-BFGS-B on f1 2-D),
 DataSet(default on f1 2-D),
 DataSet(2L on f1 2-D),
 DataSet(4L on f1 2-D),
 DataSet(8L on f1 2-D),
 DataSet(16L on f1 2-D),
 DataSet(32L on f1 2-D),
 DataSet(64L on f1 2-D),
 DataSet(portfolio-best on f2 2-D),
 DataSet(GRS on f2 2-D),
 DataSet(Powell on f2 2-D),
 DataSet(SLSQP on f2 2-D),
 DataSet(L-BFGS-B on f2 2-D),
 DataSet(default on f2 2-D),
 DataSet(2L on f2 2-D),
 DataSet(4L on f2 2-D),
 DataSet(8L on f2 2-D),
 DataSet(16L on f2 2-D),
 DataSet(32L on f2 2-D),
 DataSet(64L on f2 2-D),
 DataSet(portfolio-best on f3 2-D),
 DataSet(GRS on f3 2-D),
 DataSet(Powell on f3 2-D),
 DataSet(SLSQP on f3 2-D),
 DataSet(L-BFGS-B on f3 2-D),
 DataSet(default on f3 2-D),
 DataSet(2L on f3 2-D),
 DataSet(4L on f3 2-D),
 DataSet(8L on f3 2-D),
 DataSet(16L on f3 2-D),
 DataSet(32L on f3 2-D),
 DataSet(64L on f3 2-D),
 DataSet(portfolio-best on f4 2-D)

## Create ideal portfolio algorithm

In [207]:
import cocopp.bestalg

individual_algos = ["exdata/Powell", "exdata/SLSQP", "exdata/L-BFGS-B", "exdata/default", "exdata/2L", "exdata/4L", "exdata/8L", "exdata/16L", "exdata/32L", "exdata/64L"]
cocopp.bestalg.custom_generate(["exdata/GRS", "exdata/GRS-LOPO"] + individual_algos, algId = "VBS")

  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/GRS
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/GRS-LOPO
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/Powell
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/SLSQP
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/L-BFGS-B
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/default
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/2L
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/4L
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/8L
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/16L
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/32L
  Data consisten

In [215]:
cocopp.main(["--include-fonts", "VBS", "exdata/GRS"] + individual_algos)

Post-processing (2+)
  Using 12 data sets:
    VBS
    exdata/GRS
    exdata/Powell
    exdata/SLSQP
    exdata/L-BFGS-B
    exdata/default
    exdata/2L
    exdata/4L
    exdata/8L
    exdata/16L
    exdata/32L
    exdata/64L

Post-processing (2+)
  loading data...
  Data consistent according to consistency_check() in pproc.DataSet
  using: VBS
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/GRS
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/Powell
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/SLSQP
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/L-BFGS-B
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/default
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/2L
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/4L
  Data consistent accord

DictAlg([(('VBS', ''),
          [DataSet(VBS on f1 2-D),
           DataSet(VBS on f2 2-D),
           DataSet(VBS on f3 2-D),
           DataSet(VBS on f4 2-D),
           DataSet(VBS on f5 2-D),
           DataSet(VBS on f6 2-D),
           DataSet(VBS on f7 2-D),
           DataSet(VBS on f8 2-D),
           DataSet(VBS on f9 2-D),
           DataSet(VBS on f10 2-D),
           DataSet(VBS on f11 2-D),
           DataSet(VBS on f12 2-D),
           DataSet(VBS on f13 2-D),
           DataSet(VBS on f14 2-D),
           DataSet(VBS on f15 2-D),
           DataSet(VBS on f16 2-D),
           DataSet(VBS on f17 2-D),
           DataSet(VBS on f18 2-D),
           DataSet(VBS on f19 2-D),
           DataSet(VBS on f20 2-D),
           DataSet(VBS on f21 2-D),
           DataSet(VBS on f22 2-D),
           DataSet(VBS on f23 2-D),
           DataSet(VBS on f24 2-D),
           DataSet(VBS on f1 3-D),
           DataSet(VBS on f2 3-D),
           DataSet(VBS on f3 3-D),
           DataSe

In [217]:
cocopp.main(["--include-fonts", "VBS", "exdata/GRS", "HCMA!", "bipop-cma!"])

Post-processing (2+)
  Using 4 data sets:
    VBS
    exdata/GRS
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2013/HCMA_loshchilov_noiseless.tgz
    /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BIPOP-CMA-ES_hansen_noiseless.tgz

Post-processing (2+)
  loading data...
  Data consistent according to consistency_check() in pproc.DataSet
  using: VBS
  Data consistent according to consistency_check() in pproc.DataSet
  using: exdata/GRS
  using pickled DataSetList   using: /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2013/HCMA_loshchilov_noiseless.tgz
  using pickled DataSetList   using: /Users/lennartschapermeier/Library/Caches/cocopp/das/da/bbob/2009/BIPOP-CMA-ES_hansen_noiseless.tgz
  Will generate output data in folder ppdata/VBS_GRS_HCMA__BIPOP_6be4
    this might take several minutes.
ECDF graphs per noise group...
Loading best algorithm data from refalgs/best2009-bbob.tar.gz ...
  using pickled DataSetList   using: /Users/lenn

DictAlg([(('VBS', ''),
          [DataSet(VBS on f1 2-D),
           DataSet(VBS on f2 2-D),
           DataSet(VBS on f3 2-D),
           DataSet(VBS on f4 2-D),
           DataSet(VBS on f5 2-D),
           DataSet(VBS on f6 2-D),
           DataSet(VBS on f7 2-D),
           DataSet(VBS on f8 2-D),
           DataSet(VBS on f9 2-D),
           DataSet(VBS on f10 2-D),
           DataSet(VBS on f11 2-D),
           DataSet(VBS on f12 2-D),
           DataSet(VBS on f13 2-D),
           DataSet(VBS on f14 2-D),
           DataSet(VBS on f15 2-D),
           DataSet(VBS on f16 2-D),
           DataSet(VBS on f17 2-D),
           DataSet(VBS on f18 2-D),
           DataSet(VBS on f19 2-D),
           DataSet(VBS on f20 2-D),
           DataSet(VBS on f21 2-D),
           DataSet(VBS on f22 2-D),
           DataSet(VBS on f23 2-D),
           DataSet(VBS on f24 2-D),
           DataSet(VBS on f1 3-D),
           DataSet(VBS on f2 3-D),
           DataSet(VBS on f3 3-D),
           DataSe